# Practical Part

Implement a small app that uses all concepts introduced in the theory part.


### Exercise 1: Code Documentation Assistant
Build a system that can analyze a GitHub repository and create comprehensive documentation.

**Requirements:**
1. Create a `GitHubPlugin` that can:
   - Fetch repository contents using GitHub API
   - Extract code files with specific extensions
   - Parse directory structure

2. Create a `CodeAnalysisPlugin` that can:
   - Identify functions and classes
   - Extract comments and docstrings
   - Analyze code complexity
   - Detect dependencies

3. Use semantic memory to:
   - Store analyzed code segments
   - Remember relationships between files
   - Track dependencies
   - Store generated documentation drafts

4. Implement semantic functions for:
   - Explaining code functionality
   - Suggesting improvements
   - Creating usage examples
   - Writing test cases

5. Use the Stepwise Planner to:
   - Plan the documentation structure
   - Prioritize important components
   - Generate documentation in chunks
   - Create a final markdown document

### Exercise 2: Personal Finance Advisor
Create an AI system that can analyze financial data and provide personalized advice.

**Requirements:**
1. Create a `FinancePlugin` that can:
   - Parse CSV bank statements
   - Categorize transactions
   - Calculate spending patterns
   - Track recurring payments

2. Create a `BudgetAnalysisPlugin` that can:
   - Set budget categories
   - Compare spending to budgets
   - Identify unusual expenses
   - Project future spending

3. Use semantic memory to:
   - Store historical spending patterns
   - Remember user preferences
   - Track financial goals
   - Store advice given previously

4. Create semantic functions for:
   - Generating spending insights
   - Providing saving recommendations
   - Creating budget plans
   - Writing financial summaries

5. Use Sequential Planner to:
   - Analyze current financial state
   - Compare with historical data
   - Generate personalized advice
   - Create action plans

**Bonus:** Add integration with a financial news API to provide market context.

### Exercise 3: AI Dungeon Master
Build an intelligent game master for text-based role-playing games.

**Requirements:**
1. Create a `WorldStatePlugin` that can:
   - Track character stats
   - Manage inventory
   - Handle game world state
   - Process player actions

2. Create a `StoryGenerationPlugin` that can:
   - Generate dynamic encounters
   - Create NPCs
   - Design quests
   - Adapt story based on choices

3. Use semantic memory to:
   - Remember player choices
   - Store world history
   - Track NPC relationships
   - Save game state

4. Create semantic functions for:
   - Generating descriptions
   - Creating dialogues
   - Resolving actions
   - Adapting difficulty

5. Use Stepwise Planner to:
   - Plan story arcs
   - Generate responses to actions
   - Manage game balance
   - Create coherent narratives

**Bonus:** Add procedural content generation for locations and items.

### Exercise 5: Research Paper Assistant
Create an AI system that helps with academic research and paper writing.

**Requirements:**
1. Create a `ResearchPlugin` that can:
   - Search academic databases
   - Extract paper contents
   - Parse citations
   - Track references

2. Create a `WritingAssistantPlugin` that can:
   - Generate outlines
   - Structure arguments
   - Format citations
   - Check style guidelines

3. Use semantic memory to:
   - Store paper summaries
   - Remember key findings
   - Track citations
   - Save draft versions

4. Create semantic functions for:
   - Analyzing papers
   - Generating summaries
   - Creating literature reviews
   - Writing sections

5. Use Sequential Planner to:
   - Plan research approach
   - Organize findings
   - Structure paper
   - Generate drafts

**Bonus:** Add LaTeX support for mathematical content.

In [1]:
import json
import requests
from typing import Annotated
from bs4 import BeautifulSoup

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.memory import SemanticTextMemory
from semantic_kernel.planners import FunctionCallingStepwisePlanner, FunctionCallingStepwisePlannerOptions
from semantic_kernel.functions import kernel_function, KernelFunctionFromPrompt
from websearch import WebSearch

class WebResearchPlugin:
    """Plugin for web research capabilities"""


    @kernel_function(
        name="SearchWeb",
        description="Searches the web using DuckDuckGo and returns relevant URLs"
    )
    def search_web(
        self,
        query: Annotated[str, "a query to search the web"]
    ) -> Annotated[str, "a json result of duckduckgo search"]:
        print(f"Searching the web for: {query}")
        
        url = "https://stract.com/beta/api/search"
        headers = {
            "accept": "application/json",
            "Content-Type": "application/json",
        }
        data = {
            "query": query
        }
        response = requests.post(url, json=data, headers=headers)
        print(response)
        return response


    @kernel_function(
        name="ExtractContent",
        description="Extracts main content from a webpage"
    )
    def extract_content(
        self,
        url: Annotated[str, "the webpage URL to extract content from"]
    ) -> str:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            for element in soup(['script', 'style', 'nav', 'header', 'footer']):
                element.decompose()
            
            text = soup.get_text(separator=' ', strip=True)
            text = text[:2000] + "..." if len(text) > 2000 else text
            print(f"Extracted content: {text}")
            return text
        except Exception as e:
            return f"Error extracting content: {str(e)}"

class ResearchPlugin:
    """Plugin for analyzing and summarizing research"""
    
    def __init__(self, memory: SemanticTextMemory):
        self.memory = memory

    @kernel_function(
        name="SaveToMemory",
        description="Saves research information to semantic memory"
    )
    async def save_to_memory(
        self,
        content: Annotated[str, "the content to save to memory"],
        topic: Annotated[str, "the research topic for categorization"]
    ) -> str:
        try:
            await self.memory.save_information(
                collection="research_data",
                text=content,
                description=f"Research on {topic}",
                additional_metadata={"topic": topic}
            )
            return "Content saved to memory successfully"
        except Exception as e:
            return f"Error saving to memory: {str(e)}"

async def setup_kernel_and_memory():
    kernel = Kernel()
    
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

    embedding_service_id = "embeddings"
    kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

    memory_store = VolatileMemoryStore()
    embeddings = AzureTextEmbedding(
        service_id=embedding_service_id
    )
    memory = SemanticTextMemory(memory_store, embeddings)

    kernel.add_plugin(WebResearchPlugin(), "web")
    kernel.add_plugin(ResearchPlugin(memory), "research")

    analyze_function = KernelFunctionFromPrompt(
        function_name="AnalyzeContent",
        plugin_name="ResearchPlugin",
        prompt="""
        Content: {{$input}}
        
        Format your response as bullet points.
        """,
        description="Analyzes and extracts key points from content."
    )

    # Fixed the recall syntax in the prompt
    summarize_function = KernelFunctionFromPrompt(
        function_name="CreateSummary",
        plugin_name="ResearchPlugin",
        prompt="""{{$research_data $topic}}
        Research Topic: {{$topic}}
        Collected Information:
        
        Keep the summary clear and well-structured.
        
        """,
        description="Creates a summary from collected research."
    )

    kernel.add_function(plugin_name="ResearchPlugin", function=analyze_function)
    kernel.add_function(plugin_name="ResearchPlugin", function=summarize_function)

    return kernel

async def conduct_research(kernel: Kernel, task: str):
    planner = FunctionCallingStepwisePlanner(
        service_id="default",
        options=FunctionCallingStepwisePlannerOptions(
            max_iterations=15,
            max_tokens=4000,
        )
    )

    

    try:
        print(f"\nResearch Task: {task}\n")
        print("Starting research process...")
        
        result = await planner.invoke(kernel, task)
     
      
        print("\nResearch Results:")
        print(result.final_answer)
        
        print("\nThought Process:")
        for thought in result.chat_history:
            print(f"- {thought}")
            
    except Exception as e:
        print(f"Error during research: {str(e)}")

# Example research tasks
research_tasks = [
    """
    Research the latest developments in quantum computing.
    Focus on recent breakthroughs in error correction.
    Analyze the information and create a summary.
    """,
    """
    Find information about artificial intelligence in healthcare.
    Focus on recent applications in diagnostic imaging.
    Create a summary of the findings.
    """
]

print("🔍 Web Research Assistant with Semantic Kernel 🔍\n")

# Setup kernel and memory
kernel = await setup_kernel_and_memory()

# Process each research task
for task in research_tasks:
    await conduct_research(kernel, task)
    print("\n" + "=" * 80 + "\n")

🔍 Web Research Assistant with Semantic Kernel 🔍



NameError: name 'AzureTextEmbedding' is not defined